# Create Containerlab topology from CYJS graph

Initialize parameters   

In [140]:
topology_name = "DM-Akron"

Read CYJS graph data into a dictonary and initialize networkx graph from it

In [141]:
import json
import networkx as nx

cyjs = {}
with open(topology_name + ".cyjs", 'r', encoding='utf-8') as f:
    cyjs = json.load(f)
G = nx.cytoscape_graph(cyjs)
print(G.graph)

{'name': 'DM-Akron'}


Generate clab topology from the graph G. Using this gist as inspiration https://gist.github.com/renatoalmeidaoliveira/fdb772a5a02f3cfc0b5fbe7e8b7586a2

In [142]:
clab_nodes, clab_interfaces, clab_links = [], [], []
for n in G.nodes:
    if G.nodes[n]['type'] == 'device':
        clab_nodes.append(G.nodes[n]['device'])
    elif G.nodes[n]['type'] == 'interface' and G.nodes[n]['side'] == 'a':
        clab_interfaces.append(G.nodes[n]['interface'])
        int_name = G.nodes[n]['interface']['name']
        dev_name = None
        peer_name, peer_dev_name = None, None
        for a_adj in G.adj[n].items():
            if G.nodes[a_adj[0]]['type'] == 'device':
                dev_name = G.nodes[a_adj[0]]['device']['name']
            elif G.nodes[a_adj[0]]['type'] == 'interface':
                peer_name = G.nodes[a_adj[0]]['interface']['name']
                for b_adj in G.adj[a_adj[0]].items():
                    if G.nodes[b_adj[0]]['type'] == 'device':
                        peer_dev_name = G.nodes[b_adj[0]]['device']['name']
        clab_links.append([f"{dev_name}:{int_name}", f"{peer_dev_name}:{peer_name}"])

#print(clab_nodes)
#print(clab_interfaces)
#print(clab_links)

topology = {
    'name': G.name,
    'nodes': [f"{n['name']}" for n in clab_nodes],
    'links': [f'["{l[0]}", "{l[1]}"]' for l in clab_links],
}

print(json.dumps(topology, indent=4))

{
    "name": "DM-Akron",
    "nodes": [
        "dmi01-akron-rtr01",
        "dmi01-akron-sw01"
    ],
    "links": [
        "[\"dmi01-akron-rtr01:GigabitEthernet0/1/0\", \"dmi01-akron-sw01:GigabitEthernet1/0/1\"]",
        "[\"dmi01-akron-rtr01:GigabitEthernet0/1/1\", \"dmi01-akron-sw01:GigabitEthernet1/0/2\"]"
    ]
}


Load Jinja2 template for Containerlab to run the topology through

In [143]:
from jinja2 import Environment, FileSystemLoader
env = Environment(
            loader=FileSystemLoader(f"."),
            line_statement_prefix='#'
        )
templ = env.get_template(f"clab.j2")

Run the topology through jinja2 template to get the final result

In [144]:
config = templ.render(topology)
print(config)

name: DM-Akron
topology:
    nodes:
        dmi01-akron-rtr01:
            kind: ceos
            image: ceos
        dmi01-akron-sw01:
            kind: ceos
            image: ceos
    links:
        - endpoints: ["dmi01-akron-rtr01:GigabitEthernet0/1/0", "dmi01-akron-sw01:GigabitEthernet1/0/1"]
        - endpoints: ["dmi01-akron-rtr01:GigabitEthernet0/1/1", "dmi01-akron-sw01:GigabitEthernet1/0/2"]

